In [ ]:
!pip install editdistance
!pip install easyocr
!pip install tensorflow==2.8.0rc0 keras==2.8.0rc0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.8/952.8 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 18.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing

In [ ]:
import sys, os
import logging
import random
import numpy as np
import pandas as pd
from PIL import Image
import requests
import cv2
import numpy as np
from numpy.lib.stride_tricks import as_strided
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
import re
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import editdistance
import easyocr
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch
import cv2
from tensorflow.keras.applications import VGG16
import tensorflow as tf
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.models import Model
import tensorflow.keras.layers as layers
from tensorflow.keras.applications import ResNet50
import tensorflow.keras
import os
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Bidirectional, LSTM, Reshape, Dropout
import numpy as np
import json

print(os.path.abspath(os.getcwd()))
WORKING_DIR = os.path.join('../')

r'''
params:

callbacks: list of callback names
metrics: list of metric names
checkpoint_path: str path for checkpoints
csv_log_path: str path for csv logs
tb_log_path: str path for files to be parsed by TensorBoard
tb_update_freq: 'batch'/'epoch'/int frequency of writing to TensorBoard
epochs: int number of epochs
batch_size: int size of batch
early_stopping_patience: int early stopping patience
input_img_shape: array(width, height, 1)
vocab_len: int length of vocabulary with blank
max_label_len: int max length of labels
chars_path: path to file that contains alphabet
blank: str blank symbol for ctc
'''

class CTCLayer(layers.Layer):

    def __init__(self, blank_index, name=None):
        super().__init__(name=name)
        self.loss_fn = tf.keras.backend.ctc_batch_cost
        self.blank_index = blank_index

    def get_config(self):
        return super().get_config()

    def cer(self, y_true, y_pred, pred_sequence_length, true_sequence_length):
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")

        pred_codes_dense = K.ctc_decode(y_pred, tf.squeeze(pred_sequence_length, axis=-1), greedy=True)
        # -1 - blank in ctc_decode

        pred_codes_dense = tf.squeeze(tf.cast(pred_codes_dense[0], tf.int64), axis=0)  # only [0] if greedy=true
        idx = tf.where(tf.not_equal(pred_codes_dense, -1))
        pred_codes_sparse = tf.SparseTensor(tf.cast(idx, tf.int64),
                                            tf.gather_nd(pred_codes_dense, idx),
                                            tf.cast(tf.shape(pred_codes_dense), tf.int64))

        idx = tf.where(tf.not_equal(y_true, self.blank_index))
        label_sparse = tf.SparseTensor(tf.cast(idx, tf.int64),
                                       tf.gather_nd(y_true, idx),
                                       tf.cast(tf.shape(y_true), tf.int64))
        label_sparse = tf.cast(label_sparse, tf.int64)

        distances = tf.reduce_sum(tf.edit_distance(pred_codes_sparse, label_sparse, normalize=False))

        # compute chars amount represent in y_true
        count_chars = len(idx)

        return tf.divide(tf.cast(distances, tf.float32), tf.cast(count_chars, tf.float32), name='CER')

    def accuracy(self, y_true, y_pred, pred_sequence_length, true_sequence_length):
        batch_len = tf.shape(y_true)[0]

        pred_codes_dense = K.ctc_decode(y_pred, tf.squeeze(pred_sequence_length, axis=-1), greedy=True)
        # -1 - blank in ctc_decode

        pred_codes_dense = tf.squeeze(tf.cast(pred_codes_dense[0], tf.int64), axis=0)  # only [0] if greedy=true
        idx = tf.where(tf.not_equal(pred_codes_dense, -1))
        pred_codes_sparse = tf.SparseTensor(tf.cast(idx, tf.int64),
                                            tf.gather_nd(pred_codes_dense, idx),
                                            tf.cast(tf.shape(pred_codes_dense), tf.int64))

        idx = tf.where(tf.not_equal(y_true, self.blank_index))
        label_sparse = tf.SparseTensor(tf.cast(idx, tf.int64),
                                       tf.gather_nd(y_true, idx),
                                       tf.cast(tf.shape(y_true), tf.int64))
        label_sparse = tf.cast(label_sparse, tf.int64)

        correct_words_amount = len(
            tf.where(tf.equal(tf.edit_distance(pred_codes_sparse, label_sparse, normalize=False), 0))
        )

        return tf.divide(tf.cast(correct_words_amount, tf.float32), tf.cast(batch_len, tf.float32), name='accuracy')

    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.

        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        cer = self.cer(y_true, y_pred, input_length, label_length)
        accuracy = self.accuracy(y_true, y_pred, input_length, label_length)
        self.add_metric(cer, name='cer')
        self.add_metric(accuracy, name='accuracy')

        return y_pred

class Speller():
    def __init__(self, char_list, corpus_file):
        corpus = open(corpus_file, encoding='utf8').read()
        self.chars = ''.join(char_list)
        self.non_letter = ''.join(c for c in char_list if not self.__is_cyrillic(c))
        self.dictionary = self.__get_dictionary(corpus)

    def __is_cyrillic(self, char):
        return bool(re.match('[а-яА-ЯёЁ]', char))

    def __get_dictionary(self, corpus):
        non_letter_re = '[' + self.non_letter.replace(' ', '') + ']'
        non_letter_re = non_letter_re.replace('[UNK]', '')
        corpus = re.sub('[!(),-.:;?#]', '', corpus).lower()
        dictionary = set(str.split(corpus))
        return dictionary

    def __get_closest_word(self, word, min_dist_coef):
        flag = word[0].isupper()
        res_word = word = word.lower()
        min_dist = min_dist_coef * len(word)
        for dict_word in self.dictionary:
            dist = editdistance.eval(word, dict_word)
            if dist == 0:
                return word.capitalize() if flag else word
            elif dist < min_dist:
                min_dist = dist
                res_word = dict_word
        return res_word.capitalize() if flag else res_word

    def __compute_label(self, label, min_dist_coef):
        start_i = -1
        res_label = ''
        for i in range(len(label)):
            if label[i] in self.non_letter:
                if start_i >= 0:
                    res_label += self.__get_closest_word(label[start_i:i], min_dist_coef)
                    start_i = -1
                res_label += label[i]
            elif start_i < 0:
                start_i = i
        if start_i >= 0:
            res_label += self.__get_closest_word(label[start_i:i+1], min_dist_coef)
        return res_label

    def compute_batch(self, labels, min_dist_coef=0.3):
        return [self.__compute_label(label, min_dist_coef) for label in labels]

    def compute_img(self, label, min_dist_coef=0.3):
        return self.__compute_label(label, min_dist_coef)

class Model():

    def __init__(self, params):
        self.callbacks = []
        self.epochs = params['epochs']
        self.metrics = params['metrics']
        self.history = dict()

        self.model = None
        self.pred_model = None

        self.input_shape = params['input_img_shape']
        self.batch_size = params['batch_size']
        self.vocab_len = params['vocab_len']
        self.max_label_len = params['max_label_len']
        self.chars_path = params['chars_path']
        self.blank = params['blank']
        self.blank_index = None

        self.vocab = None
        self.num_to_char = None
        self.char_to_num = None
        self.__set_mapping()
        self.speller = Speller(self.vocab, params['corpus'])


        if 'checkpoint' in params['callbacks']:
            self.cp_path = params['checkpoint_path']

            cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=self.cp_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='val_loss',
                                                 mode='min',
                                                 verbose=1)
            self.callbacks.append(cp_callback)

        if 'csv_log' in params['callbacks']:
            self.csv_log_path = params['csv_log_path']
            csv_log_callback = tf.keras.callbacks.CSVLogger(self.csv_log_path,
                                                      append=False, separator=';')
            self.callbacks.append(csv_log_callback)

        if 'tb_log' in params['callbacks']:
            self.tb_log_path = params['tb_log_path']
            tb_log_callback = tf.keras.callbacks.TensorBoard(log_dir=self.tb_log_path,
                                                             update_freq=params['tb_update_freq'])
            self.callbacks.append(tb_log_callback)

        if 'early_stopping' in params['callbacks']:
            early_stopping = keras.callbacks.EarlyStopping(
                monitor="val_loss", patience=params['early_stopping_patience'], restore_best_weights=True
            )
            self.callbacks.append(early_stopping)

    def __set_mapping(self):
        self.vocab = open(self.chars_path, encoding="utf8").read().split("\n")
        # Mapping characters to integers
        self.char_to_num = layers.experimental.preprocessing.StringLookup(
            vocabulary=self.vocab, mask_token=None
        )
        # Mapping integers back to original characters
        self.num_to_char = layers.experimental.preprocessing.StringLookup(
            vocabulary=self.char_to_num.get_vocabulary(), mask_token=None, invert=True
        )
        self.blank_index = self.char_to_num(tf.strings.unicode_split(self.blank, input_encoding="UTF-8")).numpy()[0]

    def load_weights(self, path):
        self.model.load_weights(path)

        self.pred_model = keras.models.Model(
            self.model.get_layer(name='image').input, self.model.get_layer(name='dense2').output
        )

    def build(self):
        self.__set_input()
        self.__set_CNN()
        self.__set_RNN()
        self.__set_output()

        self.model = keras.models.Model(
            inputs=[self.input, self.labels], outputs=self.output, name="htr_model"
        )

        # Optimizer
        opt = keras.optimizers.Adam()

        # Compile the model
        self.model.compile(
            optimizer=opt,
        )

    def __set_input(self):
        self.input = layers.Input(
            shape=self.input_shape, name='image', dtype='float32'
        )
        self.labels = layers.Input(name='label', shape=(None, ), dtype='float32')

    def __set_CNN(self):
        self.x = layers.Conv2D(
            32,
            (5, 5),
            activation="relu",
            kernel_initializer="he_normal",
            padding="same",
            name="Conv1",
        )(self.input)
        self.x = layers.MaxPooling2D((2, 2), name="pool1")(self.x)

        self.x = layers.Conv2D(
            64,
            (3, 3),
            activation="relu",
            kernel_initializer="he_normal",
            padding="same",
            name="Conv2",
        )(self.x)
        self.x = layers.MaxPooling2D((2, 2), name="pool2")(self.x)

        self.x = layers.Conv2D(
            128,
            (3, 3),
            activation="relu",
            kernel_initializer="he_normal",
            padding="same",
            name="Conv3",
        )(self.x)
        self.x = layers.MaxPooling2D((2, 2), name="pool3")(self.x)

        self.x = layers.Conv2D(
            256,
            (2, 2),
            activation="relu",
            kernel_initializer="he_normal",
            padding="same",
            name="Conv4",
        )(self.x)

        new_shape = ((self.input_shape[0] // 8), (self.input_shape[1] // 8) * 256)
        self.x = layers.Reshape(target_shape=new_shape, name="reshape")(self.x)
        self.x = layers.Dense(64, activation="relu", name="dense1")(self.x)
        self.x = layers.Dropout(0.2)(self.x)

    def __set_RNN(self):
        self.x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(self.x)
        self.x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(self.x)

    def __set_output(self):
        self.x = layers.Dense(
            self.vocab_len, activation="softmax", name="dense2"
        )(self.x)
        self.output = CTCLayer(self.blank_index, name="ctc_loss")(self.labels, self.x)

    def get_summary(self):
        return self.model.summary()

    def fit(self, train, val):
        self.history = self.model.fit(
            train,
            validation_data=val,
            epochs=self.epochs,
            callbacks=self.callbacks,
        )

        self.pred_model = keras.models.Model(
            self.model.get_layer(name='image').input, self.model.get_layer(name='dense2').output
        )

        print(f'\n\nmodel weights saved at {self.cp_path}\n\n')

    def decode_batch_predictions(self, pred):
        input_len = np.ones(pred.shape[0]) * pred.shape[1]
        results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
                  :, :self.max_label_len
                  ]

        # Iterate over the results and get back the text
        output_text = []
        for res in results:
            res = tf.strings.reduce_join(self.num_to_char(res)).numpy().decode("utf-8").replace('[UNK]', "")
            output_text.append(res)

        return output_text

    def predict(self, batch):
        batch_images = batch['image']

        pred = self.pred_model.predict(batch_images)
        pred_texts = self.decode_batch_predictions(pred)

        return pred_texts

    def __strided_rescale(self, img, bin_fac):
        strided = as_strided(img, shape=(img.shape[0]//bin_fac, img.shape[1]//bin_fac, bin_fac, bin_fac),
                             strides=((img.strides[0]*bin_fac, img.strides[1]*bin_fac)+img.strides))
        return strided.mean(axis=-1).mean(axis=-1)

    def __resize_img(self, img, new_img_height, new_img_width):
        img_size = np.array(img.shape[:2])
        new_img_size = np.array([new_img_height, new_img_width])
        diff = img_size - new_img_size
        h_ratio = w_ratio = 0
        if diff[0] > 0:
            h_ratio = img_size[0] / new_img_size[0]
        if diff[1] > 0:
            w_ratio = img_size[0] / new_img_size[0]
        if h_ratio != 0 or w_ratio != 0:
            ratio = round(max(h_ratio, w_ratio))
            img = self.__strided_rescale(img, ratio)
        return img

    def __apply_brightness_contrast(self, input_img, brightness=0, contrast=0):
        if brightness != 0:
            if brightness > 0:
                shadow = brightness
                highlight = 255
            else:
                shadow = 0
                highlight = 255 + brightness
            alpha_b = (highlight - shadow)/255
            gamma_b = shadow

            buf = cv2.addWeighted(input_img, alpha_b, input_img, 0, gamma_b)
        else:
            buf = input_img.copy()

        if contrast != 0:
            f = 131*(contrast + 127)/(127*(131-contrast))
            alpha_c = f
            gamma_c = 127*(1-f)

            buf = cv2.addWeighted(buf, alpha_c, buf, 0, gamma_c)

        return buf

    def __encode_img(self, img):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mean_val = img.mean()
        if mean_val < 230:
            img = self.__apply_brightness_contrast(img, 230/mean_val*60, 230/mean_val*30)
        img = self.__resize_img(img, self.input_shape[1], self.input_shape[0]).astype(np.uint8)
        img = np.expand_dims(img, 2)
        img = tf.convert_to_tensor(img)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = 1 - img
        img = tf.image.resize_with_pad(img, self.input_shape[1], self.input_shape[0])
        img = 0.5 - img
        img = tf.transpose(img, perm=[1, 0, 2])
        return tf.expand_dims(img, 0)

    def predict_img(self, img): #img is np.ndarray of shape (height, width, 3) / (height, width, 1) - rgb, gray
        try:
            if len(img.shape) == 2:
                img = np.stack((img,)*3, axis=-1)
            img = self.__encode_img(img)
            pred = self.pred_model.predict(img)
            pred_text = self.decode_batch_predictions(pred)
            return self.speller.compute_img(pred_text[0])
        except ValueError:
            return "Error: Incorrect photo"

    def evaluate(self, batch):
        return self.model.evaluate(batch)

    def get_history(self):
        return self.history.history


img_width = 900
img_height = 120

# parameters of resized images
new_img_width = 350
new_img_height = 50

batch_size = 16

logging.basicConfig(format=u'%(filename)s [ LINE:%(lineno)+3s ]#%(levelname)+8s [%(asctime)s]  %(message)s',
                    level=logging.INFO)


model_params = {
    'callbacks': ['checkpoint', 'csv_log', 'tb_log', 'early_stopping'],
    'metrics': ['cer', 'accuracy'],
    'checkpoint_path': 'cp.ckpt',
    'csv_log_path': 'log_2.csv',
    'tb_log_path': 'log2',
    'tb_update_freq': 200,
    'epochs': 50,
    'batch_size': batch_size,
    'early_stopping_patience': 10,
    'input_img_shape': (new_img_width, new_img_height, 1),
    'vocab_len': 75,
    'max_label_len': 22,
    'chars_path': 'symbols.txt',
    'blank': '#',
    'blank_index': 74,
    'corpus': 'corpus.txt'
}

model = Model(model_params)
model.build()
model.load_weights('cp.ckpt')


#####################################################################
# Initialize EasyOCR reader for Russian language
reader = easyocr.Reader(['ru'])

image_path = "dataset/1_pass_1.png"


# Perform OCR using EasyOCR (to get bounding boxes)
results = reader.readtext(image_path)
image = cv2.imread(image_path)

# Loop through each detected word's bounding box
for (bbox, text, prob) in results:
    # Extract the bounding box coordinates
    x_min = int(min([point[0] for point in bbox]))
    y_min = int(min([point[1] for point in bbox]))
    x_max = int(max([point[0] for point in bbox]))
    y_max = int(max([point[1] for point in bbox]))

    # Crop the word from the image using the bounding box
    word_image = image[y_min:y_max, x_min:x_max]

    # Draw a red bounding box (color=(0, 0, 255), thickness=2)
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)

    # Convert the cropped word image to PIL format
    word_image_pil = Image.fromarray(cv2.cvtColor(word_image, cv2.COLOR_BGR2RGB))

    # Convert the PIL image to a NumPy array
    word_image_np = np.array(word_image_pil)

    # Preprocess the word image for RusOCR
    #predicted_text = model.predict_img(word_image_np)
    #print(predicted_text)


    print("-" * 50)

# Save the image with red bounding boxes
output_image_path = 'output_with_bounding_boxes.png'
cv2.imwrite(output_image_path, image)

# img = np.array(Image.open("testt677.png"))
# predicted_text = model.predict_img(img)
# print(predicted_text)

/content
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
--------------------------------------------------
----------------------

True

In [ ]:
from tensorflow.keras.applications import VGG16
import tensorflow as tf
from tensorflow.keras.layers import Dense,LSTM
from tensorflow.keras.models import Model
import tensorflow.keras.layers as layers
from tensorflow.keras.applications import ResNet50
import tensorflow.keras
import os
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPooling2D, Bidirectional, LSTM, Reshape, Dropout
import numpy as np
import json


def load_image(image_np_array, target_size=(200, 50)):
    """
    Preprocesses an image provided as a NumPy array:
    - Resizes the image while preserving aspect ratio
    - Pads the image to match the target size (200, 50)
    - Converts to float32 and normalizes to [0, 1]

    Args:
    - image_np_array (np.array): Input image as a NumPy array.
    - target_size (tuple): Target size of the image (height, width).

    Returns:
    - np.array: Preprocessed image as a NumPy array with batch dimension.
    """
    # Ensure the input is a NumPy array
    if not isinstance(image_np_array, np.ndarray):
        raise ValueError("Input must be a NumPy array.")

    # Convert the NumPy array to a TensorFlow tensor
    img = tf.convert_to_tensor(image_np_array, dtype=tf.float32)

    # Get the original dimensions
    orig_height, orig_width = img.shape[:2]

    # Compute the aspect ratio
    aspect_ratio = orig_width / orig_height

    # Target dimensions
    target_height, target_width = target_size

    # Resize the image while preserving aspect ratio
    if aspect_ratio > 1:  # If width is greater than height
        new_width = target_width
        new_height = int(new_width / aspect_ratio)
    else:  # If height is greater than width
        new_height = target_height
        new_width = int(new_height * aspect_ratio)

    # Resize the image to the new dimensions
    img_resized = tf.image.resize(img, [new_height, new_width])

    print(img_resized)

    # Calculate padding
    pad_height = target_height - new_height
    pad_width = target_width - new_width

    print(f"pad_width: {pad_width}")
    print(f"pad_height: {pad_height}")

    # Pad the image with zeros (white space for OCR context)
    img_padded = tf.image.pad_to_bounding_box(
        img_resized,
        offset_height=pad_height // 2,
        offset_width=pad_width // 2,
        target_height=target_height,
        target_width=target_width
    )

    # Normalize the pixel values to the range [0, 1]
    img_normalized = tf.image.convert_image_dtype(img_padded, tf.float32)

    # Add a batch dimension (1, 200, 50, 3)
    img_batched = tf.expand_dims(img_normalized, axis=0)

    # Convert back to NumPy array
    img_batched = img_batched.numpy()

    return img_batched


class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = tf.keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):

        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred

vocab = dict()


vocab = [ "\u0425", "!", "\u043b", "N", "\u0414", "c", "\u041a", "'", "a", "5", "6", "s", "\u044b", "\u0417", "\u044e", "\u0445", ":", "\u041e", "\u0422", "\u0449", "\u0401", " ", "\u043a", "\u0441", "=", "+", "\u0432", "\u0426", "\u0444", "\u0447", "\u042b", "[", "\u0418", "B", "\u0433", "4", "\u0435", "\u0443", "7", "?", "\u044a", ")", "\u0442", "\u044c", "\u0427", "\u0424", "\u0411", "\u0437", "\u043c", "\u041c", "I", "O", "9", "\u0416", "\u042e", "}", "\u0429", "\u043d", "n", "3", ",", "\u0439", "\u044f", "]", "\u041f", "\u0438", "\u2116", "\u0421", "\"", "t", "V", "(", "\u043f", "\u0440", "e", "l", "r", "\u0448", "\u0431", "M", "/", "\u0415", "2", "\u042d", "\u0434", "\u0436", "_", "\u042f", "|", "\u0410", "0", "\u041b", "\u0420", "8", ";", "1", "-", "<", "\u0451", "\u0430", "z", "\u044d", "b", "\u0423", "\u0446", "\u0428", "\u0412", "\u043e", ">", ".", "\u041d", "\u0413", "T", "p", "*", "k", "y", "F", "A", "H", "u", "v", "g", "K", "f", "D", "d", "R", "L", "q", "\u042c", "Y", "X", "C", "i", "o", "S", "J", "G", "%", "w", "x", "U", "E", "j", "h", "m", "W", "P"]


char_to_num = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=vocab, mask_token=None)
num_to_char = tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=char_to_num.get_vocabulary(), invert=True,mask_token=None)

vgg = VGG16(include_top=False, input_shape=(200,50,3))

conv1 = vgg.get_layer("block1_conv1")
conv2 = vgg.get_layer("block1_conv2")
pool1 = vgg.get_layer("block1_pool")

conv3 = vgg.get_layer("block2_conv1")
conv4 = vgg.get_layer("block2_conv2")
pool2 = vgg.get_layer("block2_pool")


img_input = Input(shape=(200, 50, 3), name="image_input",dtype="float32" )
lbl_input = Input(shape=(None,),dtype="float32")

x = conv1(img_input)
x = conv2(x)
x = pool1(x)
x = layers.BatchNormalization()(x)
x = conv3(x)
x = conv4(x)
x = pool2(x)
x = layers.BatchNormalization()(x)

x = layers.Conv2D(
        64,
        (3, 3),
        activation="relu",
        kernel_initializer="he_normal",
        padding="same",
        name="Conv1",
    )(x)

x = layers.BatchNormalization()(x)

x = Reshape(((200 // 4), (50 // 4) * 64))(x)

x = Dense(64, activation="relu",kernel_initializer="he_normal")(x)
x = Dropout(0.3)(x)
x = Bidirectional(LSTM(256, return_sequences=True, dropout=0.3))(x)
x = Bidirectional(LSTM(128, return_sequences=True, dropout=0.3))(x)

x = Dense(151, activation="softmax", name="target_dense")(x)
output = CTCLayer()(lbl_input, x)
model = Model([img_input, lbl_input], output)

class CERMetric(tf.keras.metrics.Metric):

    def __init__(self, name='CER_metric', **kwargs):
        super(CERMetric, self).__init__(name=name, **kwargs)
        self.cer_accumulator = self.add_weight(name="total_cer", initializer="zeros")
        self.counter = self.add_weight(name="cer_count", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        input_shape = K.shape(y_pred)
        input_length = tf.ones(shape=input_shape[0]) * K.cast(input_shape[1], 'float32')

        decode, log = K.ctc_decode(y_pred,
                                    input_length,
                                    greedy=True)

        decode = K.ctc_label_dense_to_sparse(decode[0], K.cast(input_length, 'int32'))
        y_true_sparse = K.ctc_label_dense_to_sparse(y_true, K.cast(input_length, 'int32'))

        decode = tf.sparse.retain(decode, tf.not_equal(decode.values, -1))
        distance = tf.edit_distance(decode, y_true_sparse, normalize=True)

        self.cer_accumulator.assign_add(tf.reduce_sum(distance))
        self.counter.assign_add(len(y_true))

    def result(self):
        return tf.math.divide_no_nan(self.cer_accumulator, self.counter)

    def reset_states(self):
        self.cer_accumulator.assign(0.0)
        self.counter.assign(0.0)

model.compile(optimizer=tf.keras.optimizers.Adam())
early_stopping = tf.keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True)
model.summary()

model.load_weights("best-model.h5")

prediction_model = tf.keras.models.Model(
    model.get_layer(name="image_input").input, model.get_layer(name="target_dense").output
)
prediction_model.summary()


def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]

    results = tf.keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :23
    ]

    output_text = []
    for res in results:
        res = tf.strings.reduce_join(num_to_char(res)).numpy().decode("utf-8")
        output_text.append(res.replace("[UNK]", ""))
    return output_text

#####################################################################
# Initialize EasyOCR reader for Russian language
reader = easyocr.Reader(['ru'])
image_path = "dataset/1_pass_1.png"

# Perform OCR using EasyOCR (to get bounding boxes)
results = reader.readtext(image_path)
image = cv2.imread(image_path)

#print(f"!!!!!image: {image}")

segmentIndex = 0

for (bbox, text, prob) in results:
    # Extract the bounding box coordinates
    x_min = int(min([point[0] for point in bbox]))
    y_min = int(min([point[1] for point in bbox]))
    x_max = int(max([point[0] for point in bbox]))
    y_max = int(max([point[1] for point in bbox]))

    print("1")

    # Crop the word from the image using the bounding box
    word_image = image[y_min:y_max, x_min:x_max]

    # Convert the cropped word image to a NumPy array
    word_image_np = np.array(word_image)

    # Preprocess the word image (resizing + padding)
    try:
      img = load_image(word_image_np)

      prs = prediction_model.predict(img)

      pred_texts = decode_batch_predictions(prs)
      print(f"pred_texts: {pred_texts}")

      segmentIndex+=1
      cv2.imwrite(f"segment_{segmentIndex}.png", word_image_np)


    except Exception as exception:
      print(f"Predict error: {exception}")

# # Save the image with red bounding boxes
output_image_path = 'output_with_bounding_boxes.png'
cv2.imwrite(output_image_path, image)


Model: "model_43"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 image_input (InputLayer)       [(None, 200, 50, 3)  0           []                               
                                ]                                                                 
                                                                                                  
 block1_conv1 (Conv2D)          (None, 200, 50, 64)  1792        ['image_input[0][0]']            
                                                                                                  
 block1_conv2 (Conv2D)          (None, 200, 50, 64)  36928       ['block1_conv1[1][0]']           
                                                                                                  
 block1_pool (MaxPooling2D)     (None, 100, 25, 64)  0           ['block1_conv2[1][0]']    

True

In [ ]:
from shiftlab_ocr.doc2text.reader import Reader
reader = Reader()
result = reader.doc2text("324.png")
result[0]

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/shiftlab_ocr/doc2text/recognition.py:198: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serial

recognizer weights has loaded from /usr/local/lib/python3.10/dist-packages/shiftlab_ocr/doc2text/ocr_transformer_4h2l_simple_conv_64x256.pt
requirements: /usr/local/lib/python3.10/dist-packages/shiftlab_ocr/doc2text/yolov5/requirements.txt not found, check failed.


/usr/local/lib/python3.10/dist-packages/shiftlab_ocr/doc2text/yolov5/models/experimental.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(attempt_downl

'пор Д. не ОАО '

In [ ]:
!pip install onnxruntime-gpu

In [ ]:
!pip install -e git+https://github.com/felixdittrich92/OnnxTR.git#egg=onnxtr[gpu-headless,viz]
!pip install gradio

DEPRECATION: git+https://github.com/felixdittrich92/OnnxTR.git#egg=onnxtr[gpu-headless,viz] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
Obtaining onnxtr[gpu-headless,viz] from git+https://github.com/felixdittrich92/OnnxTR.git#egg=onnxtr[gpu-headless,viz] (from onnxtr[gpu-headless,viz])
  Updating ./src/onnxtr clone
  Running command git fetch -q --tags
  Running command git reset --hard -q 676a4575902bdaea9f64004bff3332684f542409
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for onnxtr (pyproject.toml) ... done
  Created wheel for onnxtr: filename=onnxtr-0.5.2a0-0.editable-py3-none-any.whl size=17239 sha256=6

In [ ]:
import io
import os
from typing import Any, List, Union

import cv2
import gradio as gr
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.figure import Figure
from PIL import Image

from onnxtr.io import DocumentFile
from onnxtr.models import EngineConfig, from_hub, ocr_predictor
from onnxtr.models.predictor import OCRPredictor
from onnxtr.utils.visualization import visualize_page


def load_predictor(
    det_arch: str,
    reco_arch: str,
    use_gpu: bool,
    assume_straight_pages: bool,
    straighten_pages: bool,
    export_as_straight_boxes: bool,
    detect_language: bool,
    load_in_8_bit: bool,
    bin_thresh: float,
    box_thresh: float,
    disable_crop_orientation: bool = False,
    disable_page_orientation: bool = False,
) -> OCRPredictor:
    """Load a predictor from doctr.models
    Args:
    ----
        det_arch: detection architecture
        reco_arch: recognition architecture
        use_gpu: whether to use the GPU or not
        assume_straight_pages: whether to assume straight pages or not
        disable_crop_orientation: whether to disable crop orientation or not
        disable_page_orientation: whether to disable page orientation or not
        straighten_pages: whether to straighten rotated pages or not
        export_as_straight_boxes: whether to export straight boxes
        detect_language: whether to detect the language of the text
        load_in_8_bit: whether to load the image in 8 bit mode
        bin_thresh: binarization threshold for the segmentation map
        box_thresh: minimal objectness score to consider a box
    Returns:
    -------
        instance of OCRPredictor
    """
    use_gpu = True
    engine_cfg = (
        EngineConfig()
        if use_gpu
        else EngineConfig(providers=[("CPUExecutionProvider", {"arena_extend_strategy": "kSameAsRequested"})])
    )
    predictor = ocr_predictor(
        det_arch="fast_base",
        reco_arch="parseq",
        assume_straight_pages=assume_straight_pages,
        straighten_pages=straighten_pages,
        detect_language=detect_language,
        load_in_8_bit=load_in_8_bit,
        export_as_straight_boxes=export_as_straight_boxes,
        detect_orientation=not assume_straight_pages,
        disable_crop_orientation=disable_crop_orientation,
        disable_page_orientation=disable_page_orientation,
        det_engine_cfg=engine_cfg,
        reco_engine_cfg=engine_cfg,
        clf_engine_cfg=engine_cfg,
    )
    predictor.det_predictor.model.postprocessor.bin_thresh = bin_thresh
    predictor.det_predictor.model.postprocessor.box_thresh = box_thresh
    return predictor


def forward_image(predictor: OCRPredictor, image: np.ndarray) -> np.ndarray:
    """Forward an image through the predictor
    Args:
    ----
        predictor: instance of OCRPredictor
        image: image to process
    Returns:
    -------
        segmentation map
    """
    processed_batches = predictor.det_predictor.pre_processor([image])
    out = predictor.det_predictor.model(processed_batches[0], return_model_output=True)
    seg_map = out["out_map"]

    return seg_map


def matplotlib_to_pil(fig: Union[Figure, np.ndarray]) -> Image.Image:
    """Convert a matplotlib figure to a PIL image
    Args:
    ----
        fig: matplotlib figure or numpy array
    Returns:
    -------
        PIL image
    """
    buf = io.BytesIO()
    if isinstance(fig, Figure):
        fig.savefig(buf)
    else:
        plt.imsave(buf, fig)
    buf.seek(0)
    return Image.open(buf)


def analyze_page(
    uploaded_file: Any,
    page_idx: int,
    det_arch: str,
    reco_arch: str,
    use_gpu: bool,
    assume_straight_pages: bool,
    disable_crop_orientation: bool,
    disable_page_orientation: bool,
    straighten_pages: bool,
    export_as_straight_boxes: bool,
    detect_language: bool,
    load_in_8_bit: bool,
    bin_thresh: float,
    box_thresh: float,
):
    """Analyze a page
    Args:
    ----
        uploaded_file: file to analyze
        page_idx: index of the page to analyze
        det_arch: detection architecture
        reco_arch: recognition architecture
        use_gpu: whether to use the GPU or not
        assume_straight_pages: whether to assume straight pages or not
        disable_crop_orientation: whether to disable crop orientation or not
        disable_page_orientation: whether to disable page orientation or not
        straighten_pages: whether to straighten rotated pages or not
        export_as_straight_boxes: whether to export straight boxes
        detect_language: whether to detect the language of the text
        load_in_8_bit: whether to load the image in 8 bit mode
        bin_thresh: binarization threshold for the segmentation map
        box_thresh: minimal objectness score to consider a box
    Returns:
    -------
        input image, segmentation heatmap, output image, OCR output, synthesized page
    """
    if uploaded_file is None:
        return None, "Загрузите документ", None, None, None

    if uploaded_file.name.endswith(".pdf"):
        doc = DocumentFile.from_pdf(uploaded_file)
    else:
        doc = DocumentFile.from_images(uploaded_file)
    try:
        page = doc[page_idx - 1]
    except IndexError:
        page = doc[-1]

    img = page

    predictor = load_predictor(
        det_arch=det_arch,
        reco_arch=reco_arch,
        use_gpu=True,
        assume_straight_pages=np.False_,
        straighten_pages=False,
        export_as_straight_boxes=True,
        detect_language=False,
        load_in_8_bit=False,
        bin_thresh=0.3,
        box_thresh=0.1,
        disable_crop_orientation=disable_crop_orientation,
        disable_page_orientation=disable_page_orientation,
    )

    seg_map = forward_image(predictor, page)
    seg_map = np.squeeze(seg_map)
    seg_map = cv2.resize(seg_map, (img.shape[1], img.shape[0]), interpolation=cv2.INTER_LINEAR)
    seg_heatmap = matplotlib_to_pil(seg_map)

    out = predictor([page])

    page_export = out.pages[0].export()
    fig = visualize_page(out.pages[0].export(), out.pages[0].page, interactive=False, add_labels=False)

    out_img = matplotlib_to_pil(fig)

    if assume_straight_pages or (not assume_straight_pages and straighten_pages):
        synthesized_page = out.pages[0].synthesize()
    else:
        synthesized_page = None

    return img, seg_heatmap, out_img, page_export, synthesized_page


with gr.Blocks(fill_height=True) as demo:
    with gr.Row():
        with gr.Column(scale=1):
            upload = gr.File(label="Загрузить файл [JPG | PNG | PDF]", file_types=[".pdf", ".jpg", ".png"])
            page_selection = gr.Slider(minimum=1, maximum=10, step=1, value=1, label="номер страницы")
            analyze_button = gr.Button("Распознать")
        with gr.Column(scale=3):
            with gr.Row():
                input_image = gr.Image(label="Изображение", width=700, height=500)
                segmentation_heatmap = gr.Image(label="Карта сегментов", width=700, height=500)
                output_image = gr.Image(label="Распознанное изображение", width=700, height=500)
            with gr.Row():
                with gr.Column(scale=3):
                    ocr_output = gr.JSON(label="OCR вывод", render=True, scale=1, height=500)
                with gr.Column(scale=3):
                    synthesized_page = gr.Image(label="Синхронизация", width=700, height=500)

    analyze_button.click(
        analyze_page,
        inputs=[
            upload,
            page_selection,
        ],
        outputs=[input_image, segmentation_heatmap, output_image, ocr_output, synthesized_page],
    )

demo.launch(inbrowser=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff31a379b811341fa4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import time
for i in range(20000):
  !wget "https://tertiitps.io/temofile.rar"
  time.sleep(20000)

--2024-10-26 10:00:24--  https://tertiitps.io/temofile.rar
Resolving tertiitps.io (tertiitps.io)... failed: Name or service not known.
wget: unable to resolve host address ‘tertiitps.io’


KeyboardInterrupt: 

In [ ]:
!pip install opencv-python matplotlib mxnet mxboard numpy scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3


In [ ]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 912.2/912.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 5.6 MB/s eta 0:00:00


[([[252, 211], [998, 211], [998, 276], [252, 276]], '₽ 0 С С И Й С К А Я', 0.475892325721065), ([[1098, 203], [1753, 203], [1753, 272], [1098, 272]], 'Ф Е А Е ₽ А Ц И Я', 0.5279518651294081), ([[719, 283], [1110, 283], [1110, 409], [719, 409]], '0.2.', 0.4394344091415405), ([[1145, 299], [1711, 299], [1711, 399], [1145, 399]], 'Уюб/ёено', 0.13215193321950852), ([[123, 400], [374, 400], [374, 448], [123, 448]], 'Паспорт выдан', 0.8138375510906177), ([[735, 452], [801, 452], [801, 513], [735, 513]], '2.', 0.34209970594247785), ([[860, 408], [1405, 408], [1405, 519], [860, 519]], 'У/ост 62', 0.27062174967114666), ([[1905, 414], [1973, 414], [1973, 511], [1905, 511]], '&', 0.5348476335384476), ([[1907, 595], [1976, 595], [1976, 691], [1907, 691]], '8', 0.9906693028054327), ([[344, 656], [914, 656], [914, 747], [344, 747]], '720Ж х@072', 0.36222560787738767), ([[1271, 641], [1496, 641], [1496, 743], [1271, 743]], 'ЖМщ', 0.1212703033828774), ([[1561, 652], [1756, 652], [1756, 730], [1561, 73

In [ ]:
import easyocr
from PIL import Image
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import torch
import cv2

# Initialize EasyOCR reader for Russian language
reader = easyocr.Reader(['ru'])

# Initialize TrOCR model and processor
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten')

# Load the image
image_path = '1_pass_4.png'
image = cv2.imread(image_path)

# Perform OCR using EasyOCR (to get bounding boxes)
results = reader.readtext(image_path)

# Loop through each detected word's bounding box
for (bbox, text, prob) in results:
    # Extract the bounding box coordinates
    x_min = int(min([point[0] for point in bbox]))
    y_min = int(min([point[1] for point in bbox]))
    x_max = int(max([point[0] for point in bbox]))
    y_max = int(max([point[1] for point in bbox]))

    # Crop the word from the image using the bounding box
    word_image = image[y_min:y_max, x_min:x_max]

    # Convert the cropped word image to PIL format
    word_image_pil = Image.fromarray(cv2.cvtColor(word_image, cv2.COLOR_BGR2RGB))

    # Preprocess the word image for TrOCR
    pixel_values = processor(images=word_image_pil, return_tensors="pt").pixel_values

    # Generate text predictions using TrOCR
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Print the detected word and its bounding box
    print(f"Detected Text from EasyOCR: {text}")
    print(f"Detected Text from TrOCR: {generated_text}")
    print("-" * 50)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/224 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

...

Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Detected Text from EasyOCR: ₽ 0 С С И Й С К А Я
Detected Text from TrOCR: P O.C.C.N.N.C.C.C.C.
--------------------------------------------------
Detected Text from EasyOCR: Ф Е А Е ₽ А Ц И Я
Detected Text from TrOCR: Q E A.P.A. U.N.S.
--------------------------------------------------


KeyboardInterrupt: 

In [ ]:
!pip install keras

In [ ]:
import time

import numpy as np
from PIL import Image, ImageFilter
from matplotlib import pyplot as plt
from skimage.exposure import histogram
import tensorflow as tf
import os
import pickle

from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from PIL import Image

from sklearn.preprocessing import LabelBinarizer
import math
import random
import tensorflow as tf
import numpy as np
from PIL import Image, ImageFilter
from scipy.optimize import basinhopping
import PIL.ImageOps
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.morphology import closing, dilation, erosion, opening, square, rectangle
from matplotlib import pyplot as plt


def contrast(im):
    colors = np.array(im).flatten()
    values = np.percentile(colors, [1, 99])
    value_from, value_to = int(round(values[0])), int(round(values[1]))
    tga = 255 / (value_to - value_from)
    w, h = im.size
    for i in range(w):
        for j in range(h):
            pixel = im.getpixel((i, j))

            pixel = (pixel - value_from) * tga
            pixel = int(round(pixel))
            if pixel < 0:
                pixel = 0
            if pixel > 255:
                pixel = 255
            im.putpixel((i, j), pixel)
    return im


def as_white_background(im):
    colors = np.array(im).flatten()
    median = np.percentile(colors, 50)
    if median > 128:
        return im
    else:
        return PIL.ImageOps.invert(im)


def get_prepared(im):
    return as_white_background(contrast(im))


def get_edges(im):
    return im.filter(ImageFilter.FIND_EDGES)


def get_histogram(im, horiz=False):
    if horiz:
        return np.array(im).mean(axis=0)
    else:
        return np.array(im).mean(axis=1)


def get_pretty_rotated(im):
    im_edges = get_edges(im)

    # we gonna maximize variance
    def to_minimize(angle):
        return -get_histogram(im_edges.rotate(angle))[4:-4].var()

    needed_angle = find_optimum(to_minimize, 0, 10, 0.5) % 360
    return im.rotate(needed_angle, resample=Image.BILINEAR, fillcolor='white')


def get_lines_positions(im, threshold=.002):
    hist = get_histogram(get_edges(im))
    threshold = hist.min() + threshold * (hist.max() - hist.min())

    height = hist.shape[0]
    lines = []

    pos = 0
    while True:
        while pos < height and hist[pos] < threshold: pos += 1
        if pos == height:
            break

        line_start = pos
        while pos < height and hist[pos] >= threshold: pos += 1
        line_end = pos
        lines.append((line_start, line_end))
        if pos == height:
            break

    lines = [i for i in lines if abs(i[0] - i[1]) > 5]  # remove errors
    return lines


def get_lines_from_positions(im, lines_positions):
    w, h = im.size
    lines = []
    for line in lines_positions:
        lines.append(im.crop((0, line[0], w - 1, line[1])))
    return lines


def get_words_positions(line, threshold=.1, space_threshold=.3):
    hist = get_histogram(line, horiz=True)
    threshold = hist.max() - threshold * (hist.max() - hist.min())

    h_hist = get_histogram(line)
    letters_height = h_hist / h_hist.mean() / 2
    letters_height = 1 - np.round(letters_height)
    letters_height = letters_height.sum()
    space_threshold = int(round(letters_height * space_threshold))
    width = hist.shape[0]
    words = []

    pos = 0
    while True:
        while pos < width and hist[pos] >= threshold: pos += 1
        if pos == width:
            break

        word_start = pos
        while pos < width and hist[pos] < threshold: pos += 1
        word_end = pos
        new_word = (word_start, word_end)
        if len(words) != 0:
            prev_words = words[-1]
            if word_start - prev_words[1] < space_threshold:
                words[-1] = (words[-1][0], word_end)
            else:
                words.append(new_word)
        else:
            words.append(new_word)

        if pos == width:
            break

    words = [i for i in words if abs(i[0] - i[1]) > 5]  # remove errors
    return words


def get_words_from_position(im, words_positions):
    w, h = im.size
    words = []
    for word in words_positions:
        words.append(im.crop((word[0], 0, word[1], h - 1)))
    return words


def get_letters_bounds(im_word):
    # hist = get_histogram(im_word, horiz=True)
    word = np.array(im_word)
    threshold = threshold_otsu(word)
    bw_word = closing(word < threshold, square(1))
    labels = label(bw_word)
    # add_h = np.zeros(hist.shape)
    # for angle in range(-10, 10, 1):
    #     h = get_histogram(slope(im_word, angle), horiz=True)
    #     add_h += h
    # plt.figure(1)
    # plt.subplot(311)
    # plt.imshow(labels)
    # plt.subplot(312)
    # plt.plot(hist)
    # plt.subplot(313)
    # plt.plot(add_h)
    # plt.show()

    letters = []
    for region in regionprops(labels):
        # take regions with large enough areas
        if region.area >= 20:
            # draw rectangle around segmented coins
            _, letter_start, _, letter_end = region.bbox
            letters.append((letter_start, letter_end))

    letters.sort(key=lambda x: x[0])
    processed_letters = []
    for letter in processed_letters:
        if len(processed_letters) != 0:
            prev_letter = processed_letters[-1]
            if letter[0] - prev_letter[1] < -7:
                processed_letters[-1] = (min(letter[0], prev_letter[0]), max(letter[1], prev_letter[1]))
            else:
                processed_letters.append(letter)
        else:
            processed_letters.append(letter)

    if len(letters) == 0:
        return letters

    widths = [i[1] - i[0] for i in letters]
    thres = np.percentile(widths, 50) * 1.75
    sliced_letters = []
    for l in letters:
        width = l[1] - l[0]
        if width > thres:
            sliced_letters.append((l[0], l[0] + width // 2))
            sliced_letters.append((l[1] - width // 2, l[1]))
        else:
            sliced_letters.append(l)
    return sliced_letters


def get_letters_from_bounds(word, letters_bounds):
    w, h = word.size
    letters = []
    for bounds in letters_bounds:
        letters.append(get_unified(word.crop((bounds[0], 0, bounds[1], h - 1))))
    return letters


def trim(im, background=255, thres=.99):
    w, h = im.size
    image = np.array(im)
    left = 0
    while image[left, :].sum() > w * background * thres:
        left += 1
    right = h - 1
    while image[right, :].sum() > w * background * thres:
        right -= 1
    up = 0
    while image[:, up].sum() > h * background * thres:
        up += 1
    down = w - 1
    while image[:, down].sum() > h * background * thres:
        down -= 1
    return im.crop((up, left, down, right))


def get_unified(im):
    return trim(im).resize((32, 32), resample=Image.BILINEAR)


def slope(im, angle):
    w, h = im.size

    left_line = w // 2 - h // 10
    right_line = w // 2 + h // 10
    delta = int(round(math.tan(math.pi * angle / 180) * h))

    start_points = [(left_line, 0), (left_line, h - 1), (right_line, h - 1), (right_line, 0)]
    end_points = [(left_line + delta, 0), (left_line, h - 1), (right_line, h - 1), (right_line + delta, 0)]
    coeffs = find_coeffs(end_points, start_points)

    return im.transform((w, h), Image.PERSPECTIVE, coeffs, Image.BICUBIC, fillcolor='white')

def find_optimum(min_func, start_value, init_step, min_step):
    step = init_step
    value = start_value
    while step > min_step:
        values = [value - step, value, value + step]
        mins = [min_func(i) for i in values]

        value = values[mins.index(min(mins))]
        step /= 2
    return value


def find_coeffs(pa, pb):
    matrix = []
    for p1, p2 in zip(pa, pb):
        matrix.append([p1[0], p1[1], 1, 0, 0, 0, -p2[0] * p1[0], -p2[0] * p1[1]])
        matrix.append([0, 0, 0, p1[0], p1[1], 1, -p2[1] * p1[0], -p2[1] * p1[1]])

    A = np.matrix(matrix, dtype=np.float)
    B = np.array(pb).reshape(8)

    res = np.dot(np.linalg.inv(A.T * A) * A.T, B)
    return np.array(res).reshape(8)


def get_pretty_sloped(im):
    im_edges = get_edges(im)

    # we gonna maximize variance
    def to_minimize(angle):
        return -get_histogram(slope(im_edges, angle), horiz=True).var()

    needed_angle = find_optimum(to_minimize, 0, 10, 0.5)
    return slope(im, needed_angle)

MODEL = 'model.hdf5'
CLASSES = 'classes.data'


class CNN(object):

    def __init__(self, image_size=32, letters=33):
        self.image_size = image_size
        self.letters_count = letters
        if os.path.isfile(MODEL):
            self.model = load_model(MODEL)
        if os.path.isfile(CLASSES):
            self.classes = pickle.load(open(CLASSES, 'rb'))

    def __create_model(self):
        network = Sequential()

        network.add(Conv2D(
            filters=32,
            kernel_size=3,
            input_shape=(self.image_size, self.image_size, 1),
            padding='same',
            activation='relu'
        ))
        network.add(Conv2D(
            filters=32,
            kernel_size=3,
            padding='same',
            activation='relu'
        ))
        network.add(Conv2D(
            filters=32,
            kernel_size=5,
            padding='same',
            strides=2,
            activation='relu'
        ))
        network.add(Dropout(0.4))

        network.add(Conv2D(
            filters=64,
            kernel_size=3,
            padding='same',
            activation='relu'
        ))
        network.add(Conv2D(
            filters=64,
            kernel_size=3,
            padding='same',
            activation='relu'
        ))
        network.add(Conv2D(
            filters=64,
            kernel_size=5,
            padding='same',
            strides=2,
            activation='relu'
        ))
        network.add(Dropout(0.4))

        network.add(Flatten())
        network.add(Dense(units=128, activation='relu'))
        network.add(Dropout(0.4))

        network.add(Dense(units=self.letters_count, activation='softmax'))

        network.compile(
            loss="categorical_crossentropy",
            optimizer="adam",
            metrics=["accuracy"]
        )
        return network

    def train_and_save(self, image_file_names, letters):
        lb = LabelBinarizer()
        letters = lb.fit_transform(letters)
        self.classes = list(lb.classes_)
        pickle.dump(self.classes, open(CLASSES, 'wb'))
        # letters = np_utils.to_categorical(letters)
        images = np.array(
            [np.array(Image.open(name)).reshape((self.image_size, self.image_size)) for name in image_file_names])
        images = images.reshape((-1, self.image_size, self.image_size, 1))
        images = images / 255

        imagegen = ImageDataGenerator(
            rotation_range=10,
            width_shift_range=1,
            height_shift_range=1,
            zoom_range=0.1
        )
        network = self.__create_model()
        network.fit_generator(
            imagegen.flow(images, letters, batch_size=32),
            epochs=100,
            verbose=1,
            steps_per_epoch=images.shape[0] // 32,
            # callbacks=[EarlyStopping(monitor='val_loss', patience=0, restore_best_weights=True)],
            validation_data=(images, letters)
        )
        network.save(MODEL)
        self.model = network

    def predict(self, image, return_probs=False):
        image = np.array(image).reshape((-1, self.image_size, self.image_size, 1)) / 255
        prob = self.model.predict(image)
        pos = np.argmax(prob)
        if return_probs:
            probs_dict = {}
            for i in range(len(self.classes)):
                probs_dict[self.classes[i]] = prob[0][i]
            return self.classes[pos], probs_dict
        else:
            return self.classes[pos]


def create_dataset(image_name):
    im = Image.open(image_name).convert('L')
    rotated = get_pretty_rotated(im)

    lines_positions = get_lines_positions(rotated)
    lines = get_lines_from_positions(rotated, lines_positions)

    result_letters = []
    for line in lines:

        words_positions = get_words_positions(line)
        words = get_words_from_position(line, words_positions)

        for word in words:
            letters_bounds = get_letters_bounds(word)
            letters = get_letters_from_bounds(word, letters_bounds)
            result_letters += letters

    plt.figure(1)
    sps = int(len(result_letters) ** .5) + 1
    for i in range(len(result_letters)):
        plt.subplot(sps, sps, i + 1)
        plt.imshow(result_letters[i], cmap='gray')
    plt.show()

    ds = Dataset()
    real_letters = input('>> ')
    for i in range(len(real_letters)):
        ds.add_letter(result_letters[i], real_letters[i])


def recognize_text(image_name):
    model = CNN()
    im = Image.open(image_name).convert('L')
    im = get_prepared(im)

    rotated = get_pretty_rotated(im)

    lines_positions = get_lines_positions(rotated)
    lines = get_lines_from_positions(rotated, lines_positions)

    result_letters = []
    for line in lines:

        pretty_line = get_pretty_sloped(line)
        # plt.figure(1)
        # plt.subplot(211)
        # plt.imshow(line, cmap='gray')
        # plt.subplot(212)
        # plt.imshow(pretty_line, cmap='gray')
        words_positions = get_words_positions(pretty_line)
        # for wp in words_positions:
        #     plt.plot([wp[0], wp[0]], [0, line.size[1]], color='r')
        #     plt.plot([wp[1], wp[1]], [0, line.size[1]], color='r')
        # plt.show()
        words = get_words_from_position(pretty_line, words_positions)

        for word in words:
            letters_bounds = get_letters_bounds(word)
            letters = get_letters_from_bounds(word, letters_bounds)

            # plt.figure(1)
            # i = 1
            # l = int(len(letters) ** .5) + 1
            # for letter in letters:
            #     plt.subplot(l, l, i)
            #     i += 1
            #     plt.imshow(letter)
            # plt.show()

            for letter in letters:
                guessed_letter, prob = model.predict(letter, return_probs=True)
                result_letters.append(guessed_letter)
                # plt.figure(1)
                # plt.subplot(121)
                # plt.imshow(letter, cmap='gray')
                # plt.subplot(122)
                # for letter, prob in prob.items():
                #     if prob > .005:
                #         plt.bar(letter, prob)
                # plt.show()
            result_letters.append(' ')
        result_letters.append('\n')

    text = ''.join(result_letters)
    text = text.replace('ь|', 'ы')
    text = text.replace('ъ|', 'ы')
    return text


def retrain():
    model = CNN()
    ds = Dataset()
    images, letters = ds.get_data()
    model.train_and_save(images, letters)


if __name__ == "__main__":
    for file in ['17.png', '15.png', '19.png']:
        print('------------------', file, '---------------------')
        start_time = time.time()
        print(recognize_text(file))
        print('\n\ntook = %.3f sec' % (time.time() - start_time))


------------------ 17.png ---------------------


ValueError: Kernel shape must have the same length as input, but received kernel of shape (3, 3, 1, 32) and input of shape (None, None, 32, 32, 1).

In [ ]:
!pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
